In [ ]:
using Revise
using Plots
using LinearAlgebra

# レナードジョーンズポテンシャル
$$
U(r) = 4\epsilon\left[\left(\frac{\sigma}{r}\right)^p-\left(\frac{\sigma}{r}\right)^q\right]
$$
加わる力は
$$
F(r) = -\frac{d}{dr}U(r) = 4\epsilon\left(p\frac{\sigma^{p}}{r^{p+1}}-q\frac{\sigma^{q}}{r^{q+1}}\right)
$$

In [ ]:
potential(r,ϵ,σ,p=12,q=6)=4ϵ*((σ/r)^p - (σ/r)^q)
force(r,ϵ,σ,p=12,q=6)=4ϵ/r*(p*(σ/r)^p - q*(σ/r)^q)

## 原子モデル

In [ ]:
mutable struct Atom
    x::Vector{Float64}
    v::Vector{Float64}
    m::Float64
end

### 希ガス分子に対するLennard-Jones パラメータ

|  | $\sigma$[nm] | $\epsilon$[J] | $\epsilon/k_{B}$[K] | m [kg] |
|:---:|:---:|:---:|:---:| :---:|
| Ne | 0.274 | $0.50 \times 10^{-21}$ | 36.2 |  
| Ar | 0.340 | $1.67 \times 10^{-21}$ | 121 |$6.634 \times 10^{-26}$|   
| Kr | 0.365 | $2.25 \times 10^{-21}$ | 163 |
| Xe | 0.398 | $3.20 \times 10^{-21}$ | 232 |  

### Argonの場合

In [ ]:
ϵ = 1.67
σ = 0.34
m = 6.63e-3

f_ar(r) = force(r,ϵ,σ)

# 速度Verlet
$$
a(t) = \frac{F(r(t))}{m} \\
r(t+\Delta t) = r(t) + v(t)\Delta t + \frac{1}{2}a(t)\Delta t^{2} \\
v(t+\Delta t) = v(t) + \frac{a(t)+a(t+\Delta t)}{2}\Delta t
$$

In [ ]:
function calc_acc(as,f,rng=(0.1,1.0))
    accs=[]
    for a1 in as
        acc = [0.0,0.0]
        for a2 in as
            if a1 != a2 && (rng[1]<abs(norm(a1.x-a2.x))<rng[2])
                acc += f(norm(a1.x-a2.x))/a1.m*(a1.x-a2.x)/norm(a1.x-a2.x)
            end
        end
        push!(accs,acc)
    end
    return accs 
end

function update_x!(as,accs,Δt)
    for (a, acc) in zip(as,accs)
        a.x +=  a.v*Δt + acc*Δt^2/2
    end
end

function update_v!(as,accs,acc_nexts,Δt)
    for (a, acc,acc_next) in zip(as,accs,acc_nexts)
        a.v +=  (acc+acc_next)*Δt/2
    end
end

In [ ]:
la = 0.38 # 格子定数
cycle = 3
as = [Atom([x,y],[0.0,0.0],m) for x=-0.0:la:la*cycle,y=0.0:la:la*cycle]

Δt = 1e-6
n = 1e6
samp = 1000

anim = Animation()
for i=0:n
    if i%samp == 0
        xs = [a.x[1] for a in as]
        ys = [a.x[2] for a in as]
        time = i/samp
        plt = plot(xs,ys,st=:scatter,xlims=(-0.2,1.6),ylims=(-0.2,1.6),label="",aspect_ratio=:equal,title="$time[ns]")
        frame(anim,plt)
    end
    accs = calc_acc(as,f_ar)
    update_x!(as,accs,Δt)
    acc_nexts = calc_acc(as,f_ar)
    update_v!(as,accs,acc_nexts,Δt)
end
gif(anim,"multiparticles_LJ.gif",fps=30)